# 2017GDELT

# 1.美国报道中国的新闻占美国新闻总量（报道总量百分比）

# 2.avgtone*报道总量百分比

# 3\4.美国报道中国的econ/env/主题占美国新闻总量

# 5\6.avgtone(仅主题的报道)*美国报道中国的econ/env/主题占美国新闻总量

In [ ]:
#sql
#1.提取美国报道中国的新闻占美国新闻总量（报道总量百分比）
#提取每个国家2017年报道总量
select
A.country_code,
report_code,
A.sum_report/B.sum_report as portation
from
(
select 
A.*
from
(
select 
A.country_code,
report_code,
sum(f0_) as sum_report
from 
`dotted-marking-340807.test_20210101_202103031.wangyu_202101_202105_report_nums` A
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
and date between 20170100000000 and 20171232000000
and A.country_code <> report_code
group by A.country_code,
report_code
    )A join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
)A
join(
select 
A.country_code,
sum(f0_) as sum_report
from 
`dotted-marking-340807.test_20210101_202103031.wangyu_202101_202105_report_nums` A
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
and date between 20170100000000 and 20171232000000
    and A.country_code <> report_code
group by A.country_code
)B  on A.country_code = B.country_code

In [ ]:
#2.提取每个国家2017年报道国家平均百分比*2017年对报道国家的报道总量/每个国家2017年报道总量
select
A.country_code,
report_code,
A.toneXreport/B.sum_report as toneXreport_portation
from
(
select
A.country_code,
A.report_code,
avgTone*sum_report as toneXreport
from
(
    select 
    A.*
    from
    (
    select 
    A.country_code,
    report_code,
    sum(f0_) as sum_report
    from 
    `dotted-marking-340807.test_20210101_202103031.wangyu_202101_202105_report_nums` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    and date between 20170100000000 and 20171232000000
    and A.country_code <> report_code
    group by A.country_code,
    report_code
        )A join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
)A join 
(
    select 
    A.*
    from
    (
    select 
    FLOOR(date/10000000000) as yeardate,
    A.country_code,
    report_code,
    avg(cast(avgTone as DECIMAL)) as avgTone
    from 
    `dotted-marking-340807.test_20210101_202103031.wangyu_201101_202105_avgtone` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    and report_code is not null
    and date between 20170100000000 and 20171232000000
    and A.country_code <> report_code
    group by yeardate,
    A.country_code,
    report_code
    order by
    yeardate,
    A.country_code 
    )A join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
) B on A.country_code = B.country_code and A.report_code = B.report_code 
)A
join(
select 
A.country_code,
sum(f0_) as sum_report
from 
`dotted-marking-340807.test_20210101_202103031.wangyu_202101_202105_report_nums` A
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
and date between 20170100000000 and 20171232000000
and A.country_code <> report_code
group by A.country_code
)B  on A.country_code = B.country_code

In [ ]:
#3\4.美国报道中国的econ/env/主题占美国新闻总量
select 
FLOOR(date/10000000000) as yeardate,
C.country_code,
substring(V2Locations,instr(V2Locations,'#',1,2)+1,instr(V2Locations,'#',1,3)-instr(V2Locations,'#',1,2)-1) as report_code,
avg(cast(avgTone as DECIMAL)) as avgTone,
sum(case when INSTR(V2Themes,'ECON_') != 0 then 1 else 0 end) as ECON_num,
sum(case when INSTR(V2Themes,'ENV_') != 0 then 1 else 0 end) as ENV_num,
count(*) as numreport
from
(select 
date,
country_code,
V2Locations,
substring(V2Tone,0,instr(V2Tone,',',1,1)-1) as avgTone,
V2Themes
from 
`dotted-marking-340807.test_20210101_202103031.url_to_country` C 
join `gdelt-bq.gdeltv2.gkg` D 
on C.url = D.SourceCommonName
and D.DATE between 20150100000000 and 20210532000000
)C
group by 
yeardate,
country_code, 
report_code


#仅查询17年的这些国家
select
A.country_code,
report_code,
portation 
from
    (
    select
    A.country_code,
    report_code,
    ECON_num/f0_ as portation --ENV_num
    from 
    `dotted-marking-340807.test_20210101_202103031.wangyu_201501_202105_econ_envnums` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    where yeardate = 2017
    and A.country_code <> report_code
    ) A 
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
order by
A.country_code,
report_code

In [ ]:
#5\6.avgtone(仅主题的报道)*美国报道中国的econ/env/主题占美国新闻总量
select 
FLOOR(date/10000000000) as yeardate,
C.country_code,
substring(V2Locations,instr(V2Locations,'#',1,2)+1,instr(V2Locations,'#',1,3)-instr(V2Locations,'#',1,2)-1) as report_code,
cast(avgTone as DECIMAL) as avgTone,
case when INSTR(V2Themes,'ECON_') != 0 then 1 else 0 end as ECON_num,
case when INSTR(V2Themes,'ENV_') != 0 then 1 else 0 end as ENV_num
from
(select 
date,
country_code,
V2Locations,
substring(V2Tone,0,instr(V2Tone,',',1,1)-1) as avgTone,
V2Themes
from 
`dotted-marking-340807.test_20210101_202103031.url_to_country` C 
join `gdelt-bq.gdeltv2.gkg` D 
on C.url = D.SourceCommonName
and D.DATE between 20170100000000 and 20171532000000
)C


#获取平均tone*报道总量
select
A.country_code,
A.report_code,
portation*avgTone as tone_portation
from
(
    select
    country_code,
    report_code,
    avg(avgTone) as avgTone
    from
    `dotted-marking-340807.test_20210101_202103031.wangyu_2017_econ_env_tone`
    where ECON_num = 1 --ENV_num
    and country_code <> report_code
    group by
    country_code,
    report_code
)A join
(
select
A.country_code,
report_code,
portation 
from
    (
    select
    A.country_code,
    report_code,
    ECON_num/f0_ as portation --ENV_num
    from 
    `dotted-marking-340807.test_20210101_202103031.wangyu_201501_202105_econ_envnums` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    where yeardate = 2017
    and A.country_code <> report_code
    ) A 
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
order by
A.country_code,
report_code
)B on A.country_code = B.country_code and A.report_code = B.report_code

In [ ]:
#7/8获取两主题单独的tone，以及总的avgtone
select
A.country_code,
report_code,
avgTone
from
(
select
    A.country_code,
    report_code,
    avg(avgTone) as avgTone
    from
    `dotted-marking-340807.test_20210101_202103031.wangyu_2017_econ_env_tone` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    where ECON_num = 1 --ENV_num
    and A.country_code <> A.report_code
    group by
    country_code,
    report_code
)A 
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code

#9 获取所有新闻avgtone
select
A.country_code,
report_code,
avgTone
from
(
select
    A.country_code,
    report_code,
    avg(cast(avgTone as DECIMAL)) as avgTone
    from
    `dotted-marking-340807.test_20210101_202103031.wangyu_201101_202105_avgtone` A
    join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    where date between 20170100000000 and 20171532000000
    and A.country_code <> A.report_code
    group by
    country_code,
    report_code
)A 
join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code

In [ ]:
case when INSTR(V2Themes,'ECON_') != 0 then 1 else 0 end

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import math

#加载文件名，结果名和结果文件名

file_name = 'table_1.csv'
result_name = 'portation'
result_file_name = 'table1_report_portation.csv'

file_name = 'table_2.csv'
result_name = 'toneXreport_portation'
result_file_name = 'table2_tone_report_portation.csv'

file_name = 'table_3.csv'
result_name = 'portation'
result_file_name = 'table3_econ.csv'

file_name = 'table_4.csv'
result_name = 'portation'
result_file_name = 'table4_env.csv'

file_name = 'table_5.csv'
result_name = 'tone_portation'
result_file_name = 'table5_econtone.csv'

file_name = 'table_6.csv'
result_name = 'tone_portation'
result_file_name = 'table6_envtone.csv'

file_name = 'table_7.csv'
result_name = 'avgTone'
result_file_name = 'table7_econ_avgtone.csv'

file_name = 'table_8.csv'
result_name = 'avgTone'
result_file_name = 'table8_env_avgtone.csv'

file_name = 'table_9.csv'
result_name = 'avgTone'
result_file_name = 'table9_all_avgtone.csv'

In [92]:
file_name = 'table_9.csv'
result_name = 'avgTone'
result_file_name = 'table9_all_avgtone.csv'

#获取数据
alldata = pd.read_csv(file_name)  
APEC_country = pd.read_csv('APEC_country.csv')
APEC_country.iloc[1,4] = 'China, Hong Kong SAR'
APEC_country = APEC_country[['country_code','en_name']]

#将国家名称添加
result = pd.merge(APEC_country, alldata, on='country_code',how='inner')
result = result[['en_name', 'report_code', result_name]]
result.columns=['country1', 'country_code',result_name]
result = pd.merge(APEC_country, result, on='country_code',how='inner')
result = result[['country1', 'en_name', result_name]]
result.columns=['country1', 'country2', result_name]
#去掉文莱
country_list = APEC_country['en_name'].unique()
country_list.sort()
country_list = np.delete(country_list, 1)
#国家报道另一个国家tone运算

result_frame = ['country']
for i in country_list:
    result_frame.append(i)
len(result_frame)
result_pd = pd.DataFrame(columns = result_frame)
for time in [2017]:
    for country_A in country_list:
        now_list = []
        now_list.append(country_A)
        for country_B in country_list:
            if(country_A == country_B):
                now_list.append(0)
            elif(result[(result['country1'] == country_A) & (result['country2'] == country_B)].empty == 0):
                now_list.append(result[(result['country1'] == country_A) & (result['country2'] == country_B)].iloc[0,2])
            else:
                now_list.append(-1)
        #print(len(now_list))
        result_pd.loc[len(result_pd)] = now_list
    #print(time)
    #break
result_pd.to_csv( result_file_name , index = False)

In [93]:
result_pd

,country,Australia,Canada,Chile,China,Hong Kong,Indonesia,Japan,Malaysia,Mexico,New Zealand,Peru,Philippines,Russia,Singapore,South Korea,Taiwan,Thailand,United States,Vietnam
0,Australia,0.000000,-0.134349,-0.302660,-0.850514,0.029975,-2.752092,-0.170800,-2.081833,-1.123000,-0.005883,-1.305395,-3.927880,-2.028484,-0.514915,-1.433663,-0.849629,-1.215526,-0.994189,-0.962891
1,Canada,-1.063407,0.000000,-0.399244,-0.721618,-0.909964,-3.023101,-0.562545,-2.170716,-1.278900,-0.336014,-1.065461,-3.885964,-2.061896,-0.572995,-1.455542,-0.948009,-1.383131,-1.313993,-1.729522
2,Chile,-1.475013,-0.930943,0.000000,-1.234950,-2.208506,-2.825199,-0.791495,-2.892875,-1.099416,-0.466630,-1.922191,-4.074521,-1.311526,0.176793,-1.778272,-0.663142,-1.302779,-1.575783,-2.511046
3,China,0.251533,0.133013,1.484129,0.000000,0.187964,-0.512449,-0.082250,-0.407594,-0.068219,0.587608,0.802425,-0.378853,-0.463004,0.389254,0.169051,0.621463,0.829002,-0.142730,0.386361
4,Hong Kong,0.541631,0.188038,0.899434,0.397808,0.000000,-0.677005,0.748524,-1.664427,0.233769,0.351621,0.420164,-1.021640,-0.789574,0.514740,0.528717,0.467269,0.326265,-0.031349,0.711641
5,Indonesia,-1.405063,-1.471416,0.061886,-0.966339,-2.280560,0.000000,-0.267783,-1.176877,-1.618493,-0.660653,-0.813287,-3.382774,-2.214673,-0.570822,-0.488045,-0.501890,-0.978374,-1.481620,-1.938950
6,Japan,0.132567,0.205660,-0.045886,0.152812,-0.154890,-1.095608,0.000000,-1.099785,-0.704233,0.378476,-0.371872,-1.647156,-1.340810,0.395650,0.522736,0.274927,-0.438750,-0.211911,-0.181640
7,Malaysia,-1.514821,-1.564194,-0.704976,-1.206947,-1.493623,-2.135995,-0.559816,0.000000,-3.007523,-0.119135,-2.756846,-3.645086,-2.509479,-0.741817,-1.742486,-1.126015,-1.985740,-1.658101,-1.515821
8,Mexico,-1.109501,-0.707125,-0.492875,-1.179035,-1.994703,-2.145448,-0.416784,-1.758723,0.000000,-0.031502,-1.135992,-3.926290,-1.502088,-0.427591,-1.572987,-0.972946,-1.065484,-1.481361,-2.658807
9,New Zealand,0.074973,-0.185782,-0.250894,-0.352777,-0.688627,-2.271834,-0.151261,-1.473870,-1.603575,0.000000,-1.267603,-2.857817,-1.988798,0.344801,-1.767535,0.005582,-0.970686,-1.422724,-0.548588


In [54]:
result_pd

,country,Australia,Canada,Chile,China,Hong Kong,Indonesia,Japan,Malaysia,Mexico,New Zealand,Peru,Philippines,Russia,Singapore,South Korea,Taiwan,Thailand,United States,Vietnam
0,Australia,0.075893,0.085348,0.257488,0.171867,0.119792,0.133450,0.100517,0.075183,0.059718,0.059733,0.110411,0.052130,0.142534,0.085736,0.078050,0.119935,0.090944,0.079114,0.090078
1,Canada,0.113244,0.097522,0.322142,0.146193,0.070248,0.168982,0.154587,0.233361,0.124700,0.063826,0.272279,0.099221,0.136341,0.108541,0.069497,0.174401,0.088122,0.098917,0.094726
2,Chile,0.082649,0.102986,0.073878,0.102470,0.014085,0.205782,0.081814,0.075646,0.054926,0.056093,0.100821,0.057851,0.064643,0.098765,0.031915,0.079498,0.044881,0.071165,0.024793
3,China,0.190189,0.179831,0.192519,0.121028,0.096608,0.212217,0.107106,0.113447,0.141742,0.157159,0.067712,0.117045,0.209154,0.116308,0.115162,0.108918,0.159797,0.162107,0.152294
4,Hong Kong,0.123664,0.059829,0.129172,0.113242,0.070465,0.090476,0.089594,0.063981,0.114012,0.103314,0.106585,0.073005,0.145758,0.063741,0.089140,0.082074,0.123764,0.106532,0.108805
5,Indonesia,0.094587,0.040426,0.085865,0.052132,0.052083,0.070024,0.091090,0.165661,0.074030,0.052430,0.485507,0.043984,0.106135,0.060243,0.047953,0.048097,0.125352,0.067591,0.156069
6,Japan,0.076318,0.065956,0.184477,0.085132,0.047151,0.151603,0.070385,0.118683,0.064664,0.067882,0.061358,0.090554,0.127467,0.071115,0.031074,0.123926,0.044816,0.055798,0.052615
7,Malaysia,0.120403,0.091934,0.236696,0.085100,0.051570,0.125319,0.126639,0.088966,0.103399,0.093936,0.122917,0.090121,0.184902,0.090661,0.072170,0.108549,0.090297,0.109076,0.104002
8,Mexico,0.058737,0.067700,0.074861,0.070225,0.049430,0.144507,0.062285,0.066344,0.053606,0.062078,0.091590,0.055617,0.084682,0.112263,0.041400,0.063658,0.066856,0.056221,0.084859
9,New Zealand,0.056566,0.099786,0.148876,0.131388,0.054393,0.107385,0.106882,0.104864,0.085144,0.086065,0.102317,0.081770,0.122887,0.071782,0.065755,0.142857,0.080409,0.078920,0.095353


In [94]:
country_list

array(['Australia', 'Canada', 'Chile', 'China', 'Hong Kong', 'Indonesia',
       'Japan', 'Malaysia', 'Mexico', 'New Zealand', 'Peru',
       'Philippines', 'Russia', 'Singapore', 'South Korea', 'Taiwan',
       'Thailand', 'United States', 'Vietnam'], dtype=object)